## Importing the Libraries

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer, matthews_corrcoef

## Loading the Data

In [ ]:
train = pd.read_csv(r"C:\Users\jaimi\Downloads\Biomed Hackathon\train.csv", sep='\t')
test = pd.read_csv(r"C:\Users\jaimi\Downloads\Tab_delimited_text\Tab_delimited_text\Hackathon2024.Testing.Set.Peak2Gene.Pairs.txt\Hackathon2024.Testing.Set.Peak2Gene.Pairs.txt", sep='\t')
meta_file_path = r"C:\Users\jaimi\Downloads\Tab_delimited_text\Tab_delimited_text\Hackathon2024.Meta.txt\Hackathon2024.Meta.txt"
atac_file_path = r"C:\Users\jaimi\Downloads\Tab_delimited_text\Tab_delimited_text\Hackathon2024.ATAC.txt\Hackathon2024.ATAC.txt"
rna_file_path = r"C:\Users\jaimi\Downloads\Tab_delimited_text\Tab_delimited_text\Hackathon2024.RNA.txt\Hackathon2024.RNA.txt"

# Load the metadata
meta_df = pd.read_csv(meta_file_path, sep='\t') 
atac_df = pd.read_csv(atac_file_path, sep='\t') 
rna_df  = pd.read_csv(rna_file_path, sep='\t') 

## Perfoming Dimensionality Reduction

In [ ]:
#For RNA data
rna_numeric_df = rna_df.drop(columns=['gene'])  # Drop the 'gene' column

# For ATAC data
atac_numeric_df = atac_df.drop(columns=['peak'])  # Drop the 'peak' column

# Applying PCA to RNA data
pca_rna = PCA(n_components=0.95)  # To retain 95% of the variance
rna_pca_result = pca_rna.fit_transform(rna_numeric_df)
print("Number of components for RNA:", rna_pca_result.shape[1])

# Applying PCA to ATAC data
pca_atac = PCA(n_components=0.95)  # To retain 95% of the variance
atac_pca_result = pca_atac.fit_transform(atac_numeric_df)
print("Number of components for ATAC:", atac_pca_result.shape[1])

In [ ]:
# For RNA data
rna_pca_df = pd.DataFrame(rna_pca_result, columns=[f'PC_RNA{i+1}' for i in range(rna_pca_result.shape[1])])
rna_pca_df['gene'] = rna_df['gene']

# For ATAC data
atac_pca_df = pd.DataFrame(atac_pca_result, columns=[f'PC_ATAC{i+1}' for i in range(atac_pca_result.shape[1])])
atac_pca_df['peak'] = atac_df['peak']

## Merging the new Features

In [ ]:
# Performing an inner join on the 'gene' and 'atac' column
train_df = pd.merge(train, rna_pca_df, on='gene', how='inner') 
train_df = pd.merge(train_df, atac_pca_df, on='peak', how='inner') 


test_df = pd.merge(test, rna_pca_df, on='gene', how='inner')  
test_df = pd.merge(test_df, atac_pca_df, on='peak', how='inner')

## Feature Engineering

In [ ]:
# Extract start and end positions from the "peak" column
train_df[['chromosome', 'start', 'end']] = train_df['peak'].str.extract(r'(chr[0-9XY]+)-(\d+)-(\d+)')

# Convert start and end positions to integers for calculation
train_df['start'] = train_df['start'].astype(int)
train_df['end'] = train_df['end'].astype(int)

# Calculate the genomic distance as the difference between end and start positions
train_df['genomic_distance'] = train_df['end'] - train_df['start']

In [ ]:
# Extract start and end positions from the "peak" column
test_df[['chromosome', 'start', 'end']] = test_df['peak'].str.extract(r'(chr[0-9XY]+)-(\d+)-(\d+)')

# Convert start and end positions to integers for calculation
test_df['start'] = test_df['start'].astype(int)
test_df['end'] = test_df['end'].astype(int)

# Calculate the genomic distance as the difference between end and start positions
test_df['genomic_distance'] = test_df['end'] - test_df['start']

In [ ]:
train_df = train_df.drop(columns=['chromosome', 'start', 'end']) 
test_df = test_df.drop(columns=['chromosome', 'start', 'end'])

## Prediction Modeling 

In [ ]:
# Features: all columns except 'Peak2Gene' (target) and non-numeric columns
X = train_df.drop(columns=['Peak2Gene', 'peak', 'gene', 'Pair'])

# Target: 'Peak2Gene'
y = train_df['Peak2Gene'].astype(int)  # Converting boolean to int for classification

# Initialize Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Initialize KFold
kf = KFold(n_splits=6, shuffle=True, random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 300, 500, 1000],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Define MCC as the scoring metric
mcc_scorer = make_scorer(matthews_corrcoef)

# Initialize GridSearchCV with MCC as the scoring metric
grid_search_mcc = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                               cv=kf, scoring=mcc_scorer, n_jobs=-1, verbose=2)

# Fit the model
grid_search_mcc.fit(X, y)

# Output the best parameters and the best MCC score
print("Best Parameters:", grid_search_mcc.best_params_)
print("Best MCC Score:", grid_search_mcc.best_score_)

## Fitting the best model

In [ ]:
# Best parameters from GridSearchCV
best_params = {'bootstrap': False, 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}

# Initialize the Random Forest model with the best parameters
final_rf_model = RandomForestClassifier(**best_params, random_state=42)

# Train the model on the entire training data
final_rf_model.fit(X, y)

## Predicting outputs in Test Data

In [ ]:
X_test = test_df.drop(columns=['Peak2Gene', 'peak', 'gene', 'Pair'])

# Make predictions on the test data
test_predictions = final_rf_model.predict(X_test)

In [ ]:
# If you need to save or output the predictions
test = test.drop(columns=['Peak2Gene'])
test['Peak2Gene'] = test_predictions.astype(bool)
test.to_csv('test_predictions.csv', index=False)